In [ ]:
# importando as bibliotecas
from kafka import KafkaConsumer
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import json
from IPython.display import clear_output

# Pegando dados da AWS s3

    
# geração da nuvem de palavras em tempo real
frases = ''
for messagem in consumer:
    texto = json.loads(messagem.value.decode('utf-8'))
    frases = frases + texto['tweet']
    clear_output()
    wordcloud = WordCloud(max_font_size=100, width=1520,
                          height=535).generate(frases)
    plt.figure(figsize=(16, 9))
    plt.imshow(wordcloud)
    plt.axis("off")
    plt.show()


In [ ]:
import nltk
nltk.download("stopwords")
from nltk.corpus import stopwords

In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from IPython import display
import time

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f

spark = SparkSession.builder\
    .master('local[*]')\
    .appName('wordcloud')\
    .getOrCreate()

In [ ]:
spark

In [ ]:
def trata_tweets(df):
    words = df \
        .select (f.explode(f.split(f.lower('_c0'), " ")).alias('word')) \
        .withColumn('word', f.regexp_replace('word', r'http\S+', '')) \
        .withColumn('word', f.regexp_replace('word', r'@\w+', '')) \
        .withColumn('word', f.regexp_replace('word', 'rt', '')) \
        .na.replace('', None) \
        .na.drop()
    return words


In [ ]:
stops = stopwords.words('portuguese')
stops.append('futebol') 
plt.figure(figsize=(20, 10))

while True: 
    try:
        words = spark.read.csv('./csv', encoding='utf-8')
        words = trata_tweets(words)
        rows = words.collect()
        all_words = ''
        for row in rows:
            all_words = all_words + ' ' + row['word']

        wordcloud = WordCloud(stopwords=stops,
                     background_color="black",
                     width=1920,
                     height=1080,
                     max_words=100).generate(all_words)
        plt.cla()        
        plt.axis('off')
        plt.imshow(wordcloud)
        display.display(plt.gcf())  
        display.clear_output(wait=True) 
        time.sleep(5)
    except KeyboardInterrupt:
        break